Imports


In [31]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

Load data

In [32]:
data_path = "../udataset/State_Data.csv"
df = pd.read_csv(data_path)
state = df['state']

Predict clusters

In [33]:
# Drop the state column
features = df.drop(['state'], axis=1)

start_year = 2010
end_year = 2022
df['cluster'] = pd.NA

num_clusters = 2
for year in range(start_year, end_year + 1):
    kmeans = KMeans(n_clusters=num_clusters, random_state=0)
    year_features = features[features['year'] == year].drop(['year'], axis=1)
    cluster = kmeans.fit_predict(year_features)
    df.loc[df['year'] == year, 'cluster'] = cluster
    
df

,state,year,below_poverty_line_percent,median_household_income,mean cash public assistance income (dollars),"agriculture, forestry, fishing and hunting, and mining",construction,manufacturing,wholesale trade,retail trade,...,"professional, scientific, and management, and administrative and waste management services","educational services, and health care and social assistance","arts, entertainment, and recreation, and accommodation and food services","other services, except public administration",public administration,private wage and salary workers,government workers,self-employed in own not incorporated business workers,unpaid family workers,cluster
0,alabama,2010,0.085515,0.093612,0.049790,0.010989,0.061772,0.057585,0.060606,0.062035,...,0.011102,0.033889,0.002429,0.015110,0.012459,0.151070,0.089820,0.073944,0.05,0
1,alabama,2011,0.089878,0.097041,0.057421,0.010989,0.057110,0.056296,0.055944,0.062035,...,0.011895,0.037117,0.002834,0.016484,0.013543,0.149733,0.094311,0.066901,0.05,1
2,alabama,2012,0.094241,0.097949,0.058756,0.010379,0.053613,0.054577,0.053613,0.062862,...,0.012292,0.038731,0.003644,0.016484,0.014085,0.151070,0.097305,0.059859,0.00,0
3,alabama,2013,0.098604,0.098323,0.059265,0.010379,0.050117,0.054147,0.048951,0.062035,...,0.012688,0.041420,0.004453,0.016484,0.014085,0.149733,0.098802,0.056338,0.05,0
4,alabama,2014,0.101222,0.099360,0.060600,0.010379,0.046620,0.054577,0.048951,0.062862,...,0.012688,0.041958,0.005263,0.016484,0.014626,0.153743,0.095808,0.052817,0.05,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,wyoming,2018,0.033159,0.174752,0.098563,0.069597,0.062937,0.012892,0.034965,0.053763,...,0.003172,0.048951,0.014980,0.005495,0.016251,0.092246,0.149701,0.084507,0.10,1
672,wyoming,2019,0.032286,0.181910,0.100280,0.066545,0.065268,0.012892,0.032634,0.053763,...,0.003569,0.052717,0.014170,0.008242,0.014626,0.086898,0.151198,0.095070,0.10,1
673,wyoming,2020,0.030541,0.186955,0.098754,0.063492,0.065268,0.013752,0.032634,0.052109,...,0.002379,0.057020,0.011336,0.008242,0.015168,0.081551,0.154192,0.095070,0.15,1
674,wyoming,2021,0.029668,0.197799,0.113316,0.061050,0.066434,0.013322,0.030303,0.056245,...,0.002776,0.055406,0.010526,0.009615,0.017335,0.076203,0.161677,0.098592,0.15,1


In [34]:
NEWNAME = 'State_Data_Clustered.csv'
cols = df.columns.tolist()
df = df[[cols[0]] + [cols[-1]] + cols[1:-1]]
df.to_csv('../udataset/' + NEWNAME, index=False)
print(df[(df['cluster'] == 1) & (df['year'] == 2017)]['state'])

21                   alaska
112    district of columbia
151                  hawaii
268                maryland
320             mississippi
346                 montana
411              new mexico
450            north dakota
476                oklahoma
515             puerto rico
554            south dakota
606                 vermont
619                virginia
645           west virginia
671                 wyoming
Name: state, dtype: object
